<a href="https://colab.research.google.com/github/MichalMichniak/Sieci-Neuronowe/blob/main/Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import PIL
print(PIL.__version__)

11.0.0


In [5]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision
import torch
import torch.optim as optim
from tqdm import tqdm
import wandb

device = "cuda" if torch.cuda.is_available() else "cpu"

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [6]:
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: michalmichniak2001 (michalmichniak2001-agh). Use `wandb login --relogin` to force relogin


In [7]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [8]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [9]:
print(train_dataloader.dataset.data.shape)

torch.Size([60000, 28, 28])


### Simple ANN

In [10]:
class NN:
    def __init__(self):
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(12544, 512),
            nn.ReLU(),
            nn.Linear(512, 64),
        ).to(device)

    def forward(self, x):
        return self.model(x)

    def validate_model(self, test_dataloader, device):
        self.model.eval()  # Ustaw model w tryb ewaluacji
        total = 0
        correct = 0

        with torch.no_grad():  # Wyłączenie obliczania gradientów
            for inputs, labels in test_dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs.data, 1)  # Wybór klasy z najwyższym prawdopodobieństwem

                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Validation Accuracy: {accuracy:.2f}%")
        return accuracy
        

    def train_model(self, train_dataloader, test_dataloader, num_epochs=10, learning_rate=0.001):
        # Użycie GPU jeśli dostępne
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Zdefiniowanie optymalizatora i funkcji straty
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()  # Użycie CrossEntropyLoss dla klasyfikacji

        for epoch in range(num_epochs):
            self.model.train()  # Ustaw model w tryb treningowy
            running_loss = 0.0

            for inputs, labels in tqdm(train_dataloader):
                inputs, labels = inputs.to(device), labels.to(device)

                # Zerowanie gradientów
                optimizer.zero_grad()

                # Forward pass
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)

                # Backward pass
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            # Walidacja na końcu każdej epoki
            accuracy = self.validate_model(test_dataloader, device)
            wandb.log({"acc": accuracy, "loss": running_loss / len(train_dataloader)})
            print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_dataloader):.4f}")

In [11]:
print(NN().model)

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Flatten(start_dim=1, end_dim=-1)
  (6): Linear(in_features=12544, out_features=512, bias=True)
  (7): ReLU()
  (8): Linear(in_features=512, out_features=64, bias=True)
)


In [12]:
model = NN()
model.train_model(train_dataloader, test_dataloader, num_epochs=10, learning_rate=0.001)

100%|██████████| 938/938 [00:10<00:00, 91.55it/s] 


Validation Accuracy: 88.94%
Epoch [1/10], Loss: 0.3790


100%|██████████| 938/938 [00:09<00:00, 100.98it/s]


Validation Accuracy: 89.62%
Epoch [2/10], Loss: 0.2248


100%|██████████| 938/938 [00:09<00:00, 102.49it/s]


...

Validation Accuracy: 90.79%
Epoch [8/10], Loss: 0.0325


100%|██████████| 938/938 [00:08<00:00, 104.40it/s]


Validation Accuracy: 90.46%
Epoch [9/10], Loss: 0.0257


100%|██████████| 938/938 [00:09<00:00, 101.63it/s]


Validation Accuracy: 92.12%
Epoch [10/10], Loss: 0.0227


### Autoencoder

In [13]:
class Autoenc:
    def __init__(self):
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(12544, 512),
            nn.ReLU(),
            nn.Linear(512, 12),
        ).to(device)

        self.decoder = nn.Sequential(
            nn.Linear(12, 512),
            nn.ReLU(),
            nn.Linear(512, 12544),
            nn.ReLU(),
            nn.Unflatten(1, (64, 14, 14)),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        ).to(device)

    def forward(self, x):
        return self.decoder(self.encoder(x))

    def validate_model(self, test_dataloader, device):
        self.encoder.eval()  # Ustaw model w tryb ewaluacji
        self.decoder.eval()

        running_loss = 0

        with torch.no_grad():  # Wyłączenie obliczania gradientów
            for inputs, labels in test_dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = self.forward(inputs)
                running_loss += nn.MSELoss()(outputs, inputs).item()

        meanloss = running_loss / len(test_dataloader)
        print(f"Validation Loss: {meanloss:.2f}")

    def train_model(self, train_dataloader, test_dataloader, num_epochs=10, learning_rate=0.001):
        # Użycie GPU jeśli dostępne
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Zdefiniowanie optymalizatora i funkcji straty
        optimizer = optim.Adam(list(self.encoder.parameters()) + list(self.decoder.parameters()), lr=learning_rate)
        criterion = nn.MSELoss()  # Użycie CrossEntropyLoss dla klasyfikacji

        for epoch in range(num_epochs):
            self.encoder.train()  # Ustaw model w tryb treningowy
            self.decoder.train()
            running_loss = 0.0

            for inputs, labels in tqdm(train_dataloader):
                inputs, labels = inputs.to(device), labels.to(device)

                # Zerowanie gradientów
                optimizer.zero_grad()

                # Forward pass
                outputs = self.forward(inputs)
                loss = nn.MSELoss()(outputs, inputs)

                # Backward pass
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            # Walidacja na końcu każdej epoki
            self.validate_model(test_dataloader, device)

            print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_dataloader):.4f}")

In [14]:
model = Autoenc()
model.train_model(train_dataloader, test_dataloader, num_epochs=10, learning_rate=0.001)

100%|██████████| 938/938 [00:12<00:00, 75.35it/s]


Validation Loss: 0.01
Epoch [1/10], Loss: 0.0189


100%|██████████| 938/938 [00:10<00:00, 87.59it/s] 


Validation Loss: 0.01
Epoch [2/10], Loss: 0.0120


100%|██████████| 938/938 [00:07<00:00, 119.64it/s]


...

Validation Loss: 0.01
Epoch [8/10], Loss: 0.0095


100%|██████████| 938/938 [00:12<00:00, 76.40it/s]


Validation Loss: 0.01
Epoch [9/10], Loss: 0.0093


100%|██████████| 938/938 [00:12<00:00, 74.22it/s]


Validation Loss: 0.01
Epoch [10/10], Loss: 0.0092


In [16]:
import matplotlib.pyplot as plt

random_vector = torch.randn(1, 12).to(device)

decoded_image = model.decoder(random_vector)

decoded_image = decoded_image.reshape(28, 28).detach().cpu().numpy()

plt.imshow(decoded_image, cmap="gray")
plt.show()

In [17]:
wandb.finish()

acc,▁▂▄▇▆▇▇▅▄█
loss,█▅▄▃▂▂▁▁▁▁
acc,92.12
loss,0.02275
